In [1948]:
# Dependencies 
import pandas as pd 
import numpy as np

In [1949]:
# load CSV
school_table = pd.read_csv("../Resources/Schools.csv")
students_table = pd.read_csv("../Resources/Students.csv")  

In [1950]:
# Rename school name as school
d = {'name': 'school'}
schools_df = school_table.rename(columns=d) 

In [1951]:
# District Summary Calculation
total_budget = school_table['budget'].sum()
school_count = school_table['School ID'].count()
total_students = students_table['Student ID'].count() 
average_reading_score = students_table['reading_score'].sum()/total_students
average_math_score = students_table['math_score'].sum()/total_students
passing_maths = (students_table['math_score'].loc[students_table['math_score'] >= 70].count() / total_students)*100
passing_reading = (students_table['reading_score'].loc[students_table['reading_score'] >= 70].count() / total_students)*100
overall_passing_rate = (passing_maths + passing_reading)/2


In [1952]:
# Dataframe for District Summary
data = {'Total Schools': [school_count],'Total Students': [total_students],'Total Budget': [total_budget],'Average Math Score':[average_math_score],
        'Average Reading Score':[average_reading_score],'% Passing Math':[passing_maths],'% Passing Reading':[passing_reading],'% Overall Passing Rate':[overall_passing_rate]}
df = pd.DataFrame(data, columns = ['Total Schools','Total Students','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate'])
df  

,Total Schools,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,78.985371,81.87784,74.980853,85.805463,80.393158


In [1953]:
# Merge the two school and students table data frames on the name of the school
schools_df.school = schools_df.school.astype(str)
students_table.school = students_table.school.astype(str) 
df3 = pd.merge(schools_df, students_table, on='school', how='left') 

In [1954]:
# Rename type as school type
c = {'type': 'School Type'}
merge_res = df3.rename(columns=c) 

In [1955]:
# School Summary Calculation by grouping the school and school type
group_school = merge_res.groupby(['school','School Type'])
student_count_summ = group_school['Student ID'].count()
total_budget = group_school['budget'].unique() 
total_budget = total_budget.astype(float)
budget_per_student = total_budget / student_count_summ
avg_math = group_school['math_score'].sum()/student_count_summ
avg_read = group_school['reading_score'].sum()/student_count_summ 

In [1956]:
# School summary percentage calculation by setting the index as school
reset = merge_res.set_index('school') 
pass_maths = (reset['math_score'].loc[reset['math_score'] >= 70])
maths_pass_per = (pass_maths.groupby(['school']).count()/student_count_summ)*100
pass_read = (reset['reading_score'].loc[reset['reading_score'] >= 70])
read_pass_per = (pass_read.groupby(['school']).count()/student_count_summ)*100
overall_pass_rate = (maths_pass_per + read_pass_per)/2


In [1957]:
# Dataframe for School Summary
data = {'Total Students' : student_count_summ,'Total School Budget':total_budget,'Per Student Budget':budget_per_student,'Average Math Score': avg_math,'Average Reading Score': avg_read,
       '% Pass Math':maths_pass_per,'%Pass Reading':read_pass_per,'% Overall Passing Rate':overall_pass_rate}
df2 = pd.DataFrame(data, columns = ['Total Students','Total School Budget','Per Student Budget','Average Math Score','Average Reading Score','% Pass Math','%Pass Reading','% Overall Passing Rate'])

#Including the $ sign with Budget columns
df2["Total School Budget"] = df2["Total School Budget"].map("${0:,.0f}".format)
df2["Per Student Budget"] = df2["Per Student Budget"].map("${0:,.0f}".format)
# Text Alignment
d = dict(selector="th",
    props=[('text-align', 'left')])
df2.style.set_table_styles([d]) 

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Pass Math,%Pass Reading,% Overall Passing Rate
school,School Type,,,,,,,,
Bailey High School,District,4976,"$3,124,928",$628,77.0484,81.034,66.6801,81.9333,74.3067
Cabrera High School,Charter,1858,"$1,081,356",$582,83.0619,83.9758,94.1335,97.0398,95.5867
Figueroa High School,District,2949,"$1,884,411",$639,76.7118,81.158,65.9885,80.7392,73.3639
Ford High School,District,2739,"$1,763,916",$644,77.1026,80.7463,68.3096,79.299,73.8043
Griffin High School,Charter,1468,"$917,500",$625,83.3515,83.8168,93.3924,97.139,95.2657
Hernandez High School,District,4635,"$3,022,020",$652,77.2898,80.9344,66.753,80.863,73.808
Holden High School,Charter,427,"$248,087",$581,83.8033,83.815,92.5059,96.2529,94.3794
Huang High School,District,2917,"$1,910,635",$655,76.6294,81.1827,65.6839,81.3164,73.5002
Johnson High School,District,4761,"$3,094,650",$650,77.0725,80.9664,66.0576,81.2224,73.64


In [1958]:
# Top Performing school(Passsing Rate)
df3 = df2.nlargest(5, '% Overall Passing Rate') 
d = dict(selector="th",
    props=[('text-align', 'left')])
df3.style.set_table_styles([d]) 


,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Pass Math,%Pass Reading,% Overall Passing Rate
school,School Type,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356",$582,83.0619,83.9758,94.1335,97.0398,95.5867
Thomas High School,Charter,1635,"$1,043,130",$638,83.4183,83.8489,93.2722,97.3089,95.2905
Pena High School,Charter,962,"$585,858",$609,83.8399,84.0447,94.5946,95.9459,95.2703
Griffin High School,Charter,1468,"$917,500",$625,83.3515,83.8168,93.3924,97.139,95.2657
Wilson High School,Charter,2283,"$1,319,574",$578,83.2742,83.9895,93.8677,96.5396,95.2037


In [1959]:
# Bottom Performing School(by Passing Rate)
df4 = df2.nsmallest(5, '% Overall Passing Rate') 
d = dict(selector="th",
    props=[('text-align', 'left')])
df4.style.set_table_styles([d]) 


,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Pass Math,%Pass Reading,% Overall Passing Rate
school,School Type,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363",$637,76.8427,80.7447,66.3666,80.2201,73.2933
Figueroa High School,District,2949,"$1,884,411",$639,76.7118,81.158,65.9885,80.7392,73.3639
Huang High School,District,2917,"$1,910,635",$655,76.6294,81.1827,65.6839,81.3164,73.5002
Johnson High School,District,4761,"$3,094,650",$650,77.0725,80.9664,66.0576,81.2224,73.64
Ford High School,District,2739,"$1,763,916",$644,77.1026,80.7463,68.3096,79.299,73.8043


In [1960]:
# Math Score by grade Calculation by grouping school and grade
grp_school_grade = students_table.groupby(['school','grade'])
total_stud = grp_school_grade['Student ID'].count()
stacked_m = (grp_school_grade['math_score'].sum()/total_stud)

# Returns a new level of column labels
col_m = stacked_m.unstack()
d_m = dict(selector="th",
    props=[('text-align', 'left')])
col_m.style.set_table_styles([d_m])

grade,10th,11th,12th,9th
school,,,,
Bailey High School,76.9968,77.5156,76.4922,77.0837
Cabrera High School,83.1545,82.7656,83.2775,83.0947
Figueroa High School,76.54,76.8843,77.1514,76.403
Ford High School,77.6723,76.9181,76.18,77.3613
Griffin High School,84.2291,83.8421,83.3562,82.044
Hernandez High School,77.3374,77.136,77.1866,77.4385
Holden High School,83.4298,85,82.8554,83.7874
Huang High School,75.9087,76.4466,77.2256,77.0273
Johnson High School,76.6911,77.4917,76.8632,77.1879


In [1961]:
# Reading Score by grade Calculation by grouping school and grade
stacked_r = (grp_school_grade['reading_score'].sum()/total_stud)
col_r = stacked_r.unstack()
d_r = dict(selector="th",
    props=[('text-align', 'left')])
col_r.style.set_table_styles([d_r])

grade,10th,11th,12th,9th
school,,,,
Bailey High School,80.9072,80.9456,80.9125,81.3032
Cabrera High School,84.2532,83.7884,84.288,83.6761
Figueroa High School,81.4089,80.6403,81.3849,81.1986
Ford High School,81.2627,80.4036,80.6623,80.6327
Griffin High School,83.7069,84.2881,84.0137,83.3692
Hernandez High School,80.6601,81.3961,80.8571,80.8669
Holden High School,83.3246,83.8155,84.6988,83.6772
Huang High School,81.5124,81.4175,80.306,81.2903
Johnson High School,80.7734,80.616,81.2276,81.2607


In [1962]:
# Scores by school spending by setting the bin values
bins = [0,585,615,645,675]
p = ["<$585", "$585-615", "$615-645", "$645-675"]

In [1963]:
# Data frame for scores by school spending calculated values
data = {'Total Students' : student_count_summ,'Total School Budget':total_budget,'Per Student Budget':budget_per_student,'Average Math Score': avg_math,'Average Reading Score': avg_read,
       '% Pass Math':maths_pass_per,'%Pass Reading':read_pass_per,'% Overall Passing Rate':overall_pass_rate}
df = pd.DataFrame(data, columns = ['Total Students','Total School Budget','Per Student Budget','Average Math Score','Average Reading Score','% Pass Math','%Pass Reading','% Overall Passing Rate'])


In [1964]:
pd.cut(df["Per Student Budget"],bins,labels=p).head()

school                School Type
Bailey High School    District       $615-645
Cabrera High School   Charter           <$585
Figueroa High School  District       $615-645
Ford High School      District       $615-645
Griffin High School   Charter        $615-645
Name: Per Student Budget, dtype: category
Categories (4, object): [<$585 < $585-615 < $615-645 < $645-675]

In [1965]:
# Bin values are set to the column "Per Student Budget"
df["Per Student Budget"] = pd.cut(df["Per Student Budget"],bins,labels=p)  
b = {'Per Student Budget': 'Spending Ranges per student'}
mer_res = df.rename(columns=b) 


In [1966]:
#Calculation based on the groupby spending ranges per student
school_grp = mer_res.groupby('Spending Ranges per student')
avg_math = school_grp['Average Math Score'].mean()
avg_read = school_grp['Average Reading Score'].mean()
pass_maths = school_grp['% Pass Math'].mean()
pass_reading = school_grp['%Pass Reading'].mean() 
overall_pass = (pass_maths+pass_reading)/2


In [1967]:
data = {'Average Math Score': avg_math,'Average Reading Score': avg_read,'% Pass Math':pass_maths,'%Pass Reading':pass_reading,'% Overall Passing Rate':overall_pass}
df5 = pd.DataFrame(data, columns = ['Average Math Score','Average Reading Score','% Pass Math','%Pass Reading','% Overall Passing Rate'])
b = dict(selector="th",
    props=[('text-align', 'left')])
df5.style.set_table_styles([b])


,Average Math Score,Average Reading Score,% Pass Math,%Pass Reading,% Overall Passing Rate
Spending Ranges per student,,,,,
<$585,83.4554,83.9338,93.4601,96.6109,95.0355
$585-615,83.5997,83.8852,94.2309,95.9003,95.0656
$615-645,79.0792,81.8914,75.6682,86.1066,80.8874
$645-675,76.9972,81.0278,66.1648,81.134,73.6494


In [1968]:
# Bin values for calculating Scores by school size
bin1 = [0,1000,2000,5000]
a = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
pd.cut(df["Total Students"],bin1,labels=a).head()

school                School Type
Bailey High School    District        Large (2000-5000)
Cabrera High School   Charter        Medium (1000-2000)
Figueroa High School  District        Large (2000-5000)
Ford High School      District        Large (2000-5000)
Griffin High School   Charter        Medium (1000-2000)
Name: Total Students, dtype: category
Categories (3, object): [Small (<1000) < Medium (1000-2000) < Large (2000-5000)]

In [1969]:
df["Total Students"] = pd.cut(df["Total Students"],bin1,labels=a)  
j = {'Total Students': 'School Size'}
mer_res = df.rename(columns=j)

In [1970]:
# Scores by school size is calculated by grouping 'School Size'
school_grp = mer_res.groupby('School Size')
avg_maths = school_grp['Average Math Score'].mean()
avg_reads = school_grp['Average Reading Score'].mean()
pass_math = school_grp['% Pass Math'].mean()
pass_read = school_grp['%Pass Reading'].mean() 
overall_pass_percent = (pass_math+pass_read)/2

In [1971]:
# Data frame for scores by school size calculation
data = {'Average Math Score': avg_maths,'Average Reading Score': avg_reads,'% Pass Math':pass_math,'%Pass Reading':pass_read,'% Overall Passing Rate':overall_pass_percent}
df6 = pd.DataFrame(data, columns = ['Average Math Score','Average Reading Score','% Pass Math','%Pass Reading','% Overall Passing Rate'])
d = dict(selector="th",
    props=[('text-align', 'left')])
df6.style.set_table_styles([d]) 

,Average Math Score,Average Reading Score,% Pass Math,%Pass Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.8216,83.9298,93.5502,96.0994,94.8248
Medium (1000-2000),83.3747,83.8644,93.5997,96.7907,95.1952
Large (2000-5000),77.7464,81.3445,69.9634,82.7666,76.365


In [1972]:
# Scores by school type calculation by grouping 'School Type'
school_grp = mer_res.groupby('School Type')
avg_maths_score = school_grp['Average Math Score'].mean()
avg_reads_score = school_grp['Average Reading Score'].mean()
pass_math_score = school_grp['% Pass Math'].mean()
pass_read_score = school_grp['%Pass Reading'].mean() 
overall_pass_per = (pass_math_score+pass_read_score)/2

In [1973]:
# Data frame for Scores by school Type 
data = {'Average Math Score': avg_maths_score,'Average Reading Score': avg_reads_score,'% Pass Math':pass_math_score,'%Pass Reading':pass_read_score,'% Overall Passing Rate':overall_pass_per}
df7 = pd.DataFrame(data, columns = ['Average Math Score','Average Reading Score','% Pass Math','%Pass Reading','% Overall Passing Rate'])
n = dict(selector="th",
    props=[('text-align', 'left')])
df7.style.set_table_styles([n]) 

,Average Math Score,Average Reading Score,% Pass Math,%Pass Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.4739,83.8964,93.6208,96.5865,95.1037
District,76.9567,80.9666,66.5485,80.7991,73.6738
